In [1]:
import sklearn
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.ensemble import RandomTreesEmbedding, HistGradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.model_selection import train_test_split
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, SGDRegressor, RidgeCV, Ridge
from sklearn.model_selection import ParameterGrid, ParameterSampler, RandomizedSearchCV
from sklearn.gaussian_process.kernels import RBF, RationalQuadratic
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, KernelCenterer, RobustScaler
from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, make_scorer, max_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import joblib
from sklearn.model_selection import cross_val_score, KFold, cross_validate, cross_val_predict
from sklearn.pipeline import make_pipeline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random, time
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, max_error
from sklearn.linear_model import LinearRegression

## BEFORE PRUNING

In [3]:
df_all = pd.read_csv('z_score_3_data_all.csv')
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [4]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.9999999813543439 
 Beta : 0.977405104213847 
 V : 0.9999997375615296 
MAE : 
 Alpha : 0.002604547261349629 
 Beta : 0.559077213592743 
 V : 0.0018585468821989413 
MSE : 
 Alpha : 2.4353795012336788e-05 
 Beta : 2.5045052507851886 
 V : 5.227615544605306e-05 
Max AERR : 
 V     0.084655
α     0.044422
β    15.077669
dtype: float64
Waktu yang dibutuhkan :  1.9809417009353638


In [5]:
df_all_before = pd.read_csv('z_score_3_data_all.csv')
df_test_before = df_all_before[(df_all_before["α"] >= -30) & (df_all_before["α"] <= 0) & (df_all_before["β"] <= 0) ]
print(len(df_test_before))
# Alpha Test Data
df_test_alpha = df_test_before.copy()
df_test_alpha["α"] *= -1 
df_test_alpha[['Pα1', 'Pα2']] = df_test_alpha[['Pα2', 'Pα1']]
df_test_alpha
print(len(df_test_alpha))

# Beta Test Data
df_test_beta = df_test_before.copy()
df_test_beta["β"] *= -1 
df_test_beta[['Pβ1', 'Pβ2']] = df_test_beta[['Pβ2', 'Pβ1']]
df_test_beta
print(len(df_test_beta))

# Combine Test Data
df_test_alpha_predict = df_test_alpha.sample(n = 500, random_state = 42)
df_test_beta_predict = df_test_beta.sample(n = 500, random_state = 42)
df_test_combine = pd.concat([df_test_alpha_predict,df_test_beta_predict], ignore_index = True)
df_test_combine

# Separation Data Test
x_test = df_test_combine.drop(columns = ["V","α","β"])
y_test = df_test_combine[["V","α","β"]]


1298
1298
1298


In [6]:
# Evaluate the metrics
y0_test_pred = model_all_best.predict(x_test)
r2_test = r2_score(y_test, y0_test_pred, multioutput = 'raw_values')
mae_test = mean_absolute_error(y_test, y0_test_pred, multioutput = 'raw_values')
mse_test = mean_squared_error(y_test, y0_test_pred, multioutput = 'raw_values')
max_ae_test = max_absolute_error(y_test, y0_test_pred)
print(f'R2 : \n Alpha : {r2_test[1]} \n Beta : {r2_test[2]} \n V : {r2_test[0]} ')
print(f'MAE : \n Alpha : {mae_test[1]} \n Beta : {mae_test[2]} \n V : {mae_test[0]} ')
print(f'MSE : \n Alpha : {mse_test[1]} \n Beta : {mse_test[2]} \n V : {mse_test[0]} ')
print('Max AERR : \n', max_ae_test)

R2 : 
 Alpha : 0.9992408870232796 
 Beta : 0.9886492418830061 
 V : 0.9999851601953914 
MAE : 
 Alpha : 0.06670888534747436 
 Beta : 0.5815201175977703 
 V : 0.031845454181233586 
MSE : 
 Alpha : 0.21076771798642552 
 Beta : 1.8774153925507915 
 V : 0.002862048842189777 
Max AERR : 
 V    0.315345
α    4.975338
β    7.029296
dtype: float64


## AFTER PRUNING

In [7]:
df_all = pd.read_csv('data_clean_reduce_21_z_score_3_sec.csv')
x_all = df_all.drop(columns = ["V","α","β"])
y_all = df_all[["V","α","β"]]

x0_train, x0_test, y0_train, y0_test = train_test_split(x_all, y_all,
                                                        random_state = 42, test_size = 0.2)

In [8]:
def max_absolute_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred), axis=0)
scaler = StandardScaler()

# Base models (Level Pertama)
base_models = [
    # ('random_forest', RandomForestRegressor(n_estimators=250, max_depth=None, random_state=42, min_samples_leaf= 1, min_samples_split= 2, max_features = 'sqrt')),
    ('random_forest', RandomForestRegressor(n_estimators=100, max_depth=None, random_state=21, max_leaf_nodes= 800)),
    ('gradient_boosting', GradientBoostingRegressor(n_estimators=400, learning_rate=0.0579, random_state=42, max_depth = 5)),
    ('knn', make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=2, weights='distance', algorithm='auto')))
]

# Meta-model (Level Kedua)
# Menggunakan Ridge Regression sebagai meta-model untuk menghindari overfitting
meta_model = Ridge(alpha=0.1)

# Membentuk Stacking Regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    passthrough=True  # Memasukkan data asli ke meta-model bersama prediksi base models
)

# Untuk multi-output, kita bungkus dengan MultiOutputRegressor
model_all_best = MultiOutputRegressor(stacking_regressor)
start = time.time()
# Training
model_all_best.fit(x0_train, y0_train)
end = time.time()
waktu = (end-start)/60
# Prediction
y0_pred = model_all_best.predict(x0_test)
r2 = r2_score(y0_test, y0_pred, multioutput = 'raw_values')
mae = mean_absolute_error(y0_test, y0_pred, multioutput = 'raw_values')
mse = mean_squared_error(y0_test, y0_pred, multioutput = 'raw_values')
max_ae = max_absolute_error(y0_test, y0_pred)
print(f'R2 : \n Alpha : {r2[1]} \n Beta : {r2[2]} \n V : {r2[0]} ')
print(f'MAE : \n Alpha : {mae[1]} \n Beta : {mae[2]} \n V : {mae[0]} ')
print(f'MSE : \n Alpha : {mse[1]} \n Beta : {mse[2]} \n V : {mse[0]} ')
print('Max AERR : \n', max_ae)
print("Waktu yang dibutuhkan : ", waktu)

R2 : 
 Alpha : 0.999999982615092 
 Beta : 0.9834776440134287 
 V : 0.9999997491265673 
MAE : 
 Alpha : 0.0027604065001561187 
 Beta : 0.48633491791061023 
 V : 0.0018246195926608683 
MSE : 
 Alpha : 2.161358572749577e-05 
 Beta : 1.7783882996031715 
 V : 5.0067257958282746e-05 
Max AERR : 
 V    0.073124
α    0.036472
β    9.831914
dtype: float64
Waktu yang dibutuhkan :  1.9731943408648174


In [9]:
df_all_before = pd.read_csv('data_clean_reduce_21_z_score_3_sec.csv')
df_test_before = df_all_before[(df_all_before["α"] >= -30) & (df_all_before["α"] <= 0) & (df_all_before["β"] <= 0) ]
print(len(df_test_before))
# Alpha Test Data
df_test_alpha = df_test_before.copy()
df_test_alpha["α"] *= -1 
df_test_alpha[['Pα1', 'Pα2']] = df_test_alpha[['Pα2', 'Pα1']]
df_test_alpha
print(len(df_test_alpha))

# Beta Test Data
df_test_beta = df_test_before.copy()
df_test_beta["β"] *= -1 
df_test_beta[['Pβ1', 'Pβ2']] = df_test_beta[['Pβ2', 'Pβ1']]
df_test_beta
print(len(df_test_beta))

# Combine Test Data
df_test_alpha_predict = df_test_alpha.sample(n = 500, random_state = 42)
df_test_beta_predict = df_test_beta.sample(n = 500, random_state = 42)
df_test_combine = pd.concat([df_test_alpha_predict,df_test_beta_predict], ignore_index = True)
df_test_combine

# Separation Data Test
x_test = df_test_combine.drop(columns = ["V","α","β"])
y_test = df_test_combine[["V","α","β"]]


1298
1298
1298


In [10]:
# Evaluate the metrics
y0_test_pred = model_all_best.predict(x_test)
r2_test = r2_score(y_test, y0_test_pred, multioutput = 'raw_values')
mae_test = mean_absolute_error(y_test, y0_test_pred, multioutput = 'raw_values')
mse_test = mean_squared_error(y_test, y0_test_pred, multioutput = 'raw_values')
max_ae_test = max_absolute_error(y_test, y0_test_pred)
print(f'R2 : \n Alpha : {r2_test[1]} \n Beta : {r2_test[2]} \n V : {r2_test[0]} ')
print(f'MAE : \n Alpha : {mae_test[1]} \n Beta : {mae_test[2]} \n V : {mae_test[0]} ')
print(f'MSE : \n Alpha : {mse_test[1]} \n Beta : {mse_test[2]} \n V : {mse_test[0]} ')
print('Max AERR : \n', max_ae_test)

R2 : 
 Alpha : 0.9992592514369122 
 Beta : 0.9882603814909892 
 V : 0.9999850925190209 
MAE : 
 Alpha : 0.06402932966288247 
 Beta : 0.5970592003615787 
 V : 0.03192196798542855 
MSE : 
 Alpha : 0.20566883854131757 
 Beta : 1.9417329013904003 
 V : 0.0028751011082537247 
Max AERR : 
 V    0.315681
α    4.977695
β    6.810382
dtype: float64
